In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lightning as L
import torch


from SoccerNet.Evaluation.MV_FoulRecognition import evaluate
from torch.utils.data import DataLoader, random_split

from src.training import TrainingConfig
from src.dataset import MultiViewDataset
from src.model import LitMVNNetwork, get_pre_model
from src.loss import get_criterion
from src.eval import save_evaluation_file
from datetime import datetime
import os
from src.augment import get_augmentation
import wandb
from lightning.pytorch.loggers import WandbLogger

In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adamsebastiangorski. Use `wandb login --relogin` to force relogin


True

In [4]:
torch.cuda.empty_cache()

In [5]:
num_epochs = 10
start_frame = 0
end_frame = 64
fps = 25
num_views = 5
pre_model = "s3d"
max_num_worker = 0
batch_size = 2
data_aug = False
pooling_type = 'max'
weight_decay = 0.001
step_size = 3
gamma = 0.1
LR = 0.01
weighted_loss = False
data_aug = False

training_config = TrainingConfig(start_frame=start_frame, end_frame=end_frame, fps=fps, num_views = num_views, pre_model = pre_model,
                                 max_num_worker=max_num_worker, batch_size=batch_size, data_aug=data_aug, pooling_type=pooling_type, 
                                 weight_decay=weight_decay, step_size=step_size, gamma=gamma, LR=LR, weighted_loss=weighted_loss)


In [6]:
wandb.init(
    project="ZZSN multi-view-foul-recognition",
    config=training_config.model_dump()
)

In [7]:
path = "data/mvfouls"
predictions_output_dir = "outputs"

transform_aug = get_augmentation(training_config.data_aug)
transforms_model = get_pre_model(training_config.pre_model)


In [8]:
dataset_Train = MultiViewDataset(path=path, start=start_frame, end=end_frame, fps=fps, split='Train',
                                 num_views=5, transform=transform_aug, transform_model=transforms_model)

train_size = int(0.7 * len(dataset_Train))
val_size = len(dataset_Train) - train_size

train_set, val_set = random_split(dataset_Train, [train_size, val_size])
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=max_num_worker, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=max_num_worker, pin_memory=True) 

2319


In [9]:
dataset_dir = "datasets"

dataset_Test = MultiViewDataset(path=path, start=start_frame, end=end_frame, fps=fps, split='Test', num_views = 5, 
transform_model=transforms_model)


dataset_Chall = MultiViewDataset(path=path, start=start_frame, end=end_frame, fps=fps, split='Chall', num_views = 5, 
        transform_model=transforms_model)

test_loader = torch.utils.data.DataLoader(dataset_Test,
            batch_size=1, shuffle=False,
            num_workers=max_num_worker, pin_memory=True)
        
chall_loader = torch.utils.data.DataLoader(dataset_Chall,
            batch_size=1, shuffle=False,
            num_workers=max_num_worker, pin_memory=True)

251
273


In [10]:
criterion = get_criterion(weighted_loss, dataset_train=dataset_Train)
model = LitMVNNetwork(pre_model=pre_model, pooling_type=pooling_type, criterion=criterion, config=training_config)
job_id = str(datetime.now())
wand_logger = WandbLogger(log_model="all")

In [11]:
trainer = L.Trainer(limit_train_batches=3000, limit_val_batches=50, max_epochs=num_epochs, logger=wand_logger)
trainer.fit(model=model, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wan

Epoch 0:   0%|          | 0/812 [00:00<?, ?it/s] 

/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")
/home/agorski/Desktop/projects/experiments_zzsn/.venv/lib/python3.11/site-packages/lightning/pytorch/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 0: 100%|█████████▉| 811/812 [21:44<00:01,  0.62it/s, v_num=ylwx, train_step_loss=2.400]

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
os.makedirs(predictions_output_dir, exist_ok=True)

test_set = f"test_{job_id}"
chall_set = f"chall_{job_id}"

test_prediction_file = save_evaluation_file(test_loader, model=model, set_name=test_set, output_dir=predictions_output_dir)
chall_prediction_file = save_evaluation_file(chall_loader, model=model, set_name=chall_set, output_dir=predictions_output_dir)

  0%|          | 0/251 [00:00<?, ?it/s]

100%|██████████| 273/273 [13:25<00:00,  2.95s/it]


In [ ]:
test_results = evaluate(os.path.join(path, "Test", "annotations.json"), test_prediction_file)
wandb.log(test_results)

<class 'numpy.ndarray'>


In [ ]:
wandb.finish()

accuracy_action,▁
accuracy_offence_severity,▁
balanced_accuracy_action,▁
balanced_accuracy_offence_severity,▁
epoch,▁▁▅▅██
leaderboard_value,▁
train_epoch_loss,▁█▄
trainer/global_step,▁▁▂▃▃▃▃▃▅▅▄▅▅██
val_epoch_loss,▁
val_step_loss,▁▂█
accuracy_action,17.13147
